<a href="https://colab.research.google.com/github/AIVoiceAskDeeper/work/blob/main/%E3%80%90SIGNATE%E7%AC%AC35%E5%9B%9E%E3%82%B3%E3%83%B3%E3%83%9A%E3%80%91%E5%8C%BB%E7%99%82%E4%BF%9D%E9%99%BA%E3%81%AE%E8%B2%BB%E7%94%A8%E5%B8%AF%E4%BA%88%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pandas のインポート
import pandas as pd
# numpy のインポート
import numpy as np
# matplotlib のインポート
%matplotlib inline
from matplotlib import pyplot as plt


In [ ]:
sample = pd.read_csv("sample_submit.csv")
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
combine = [train, test]
# データの中身を確認
train.head()

,id,age,sex,bmi,children,smoker,region,charges
0,0,26,male,32.665465,3,no,southeast,0
1,1,41,male,29.798725,1,no,southwest,0
2,2,28,male,32.722029,0,yes,northwest,1
3,3,20,female,38.429831,2,no,southeast,0
4,4,45,female,29.641854,1,no,northwest,0


In [ ]:
# 欠損値の確認
train.isnull().sum()

id          0
age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [ ]:
train_describe = train.describe()
train_describe

,id,age,bmi,children,charges
count,1600.00000,1600.000000,1600.000000,1600.000000,1600.000000
mean,1007.86250,38.985000,32.424376,1.014375,0.306250
std,575.26841,13.555012,5.766915,1.259031,0.628656
min,0.00000,18.000000,20.627626,0.000000,0.000000
25%,515.75000,27.000000,28.634267,0.000000,0.000000
50%,1010.50000,40.000000,32.268786,1.000000,0.000000
75%,1509.25000,50.000000,37.069581,2.000000,0.000000
max,1999.00000,64.000000,47.290644,5.000000,2.000000


In [ ]:
#!pip install pandas-profiling

In [ ]:
import pandas as pd
import pandas_profiling as pdp

profile = pdp.ProfileReport(train)

In [ ]:
#profile

In [ ]:
train_corr=train.corr()
train_corr

,id,age,bmi,children,charges
id,1.000000,0.005803,0.039221,0.012440,0.013463
age,0.005803,1.000000,0.019885,0.011519,0.102112
bmi,0.039221,0.019885,1.000000,0.083677,0.171282
children,0.012440,0.011519,0.083677,1.000000,0.026830
charges,0.013463,0.102112,0.171282,0.026830,1.000000


In [ ]:
# px_height と px_width を掛け合わせて相関をとってみる
for dataset in combine:
    dataset['bmi*children'] = dataset.bmi * (dataset.children+1)

train.loc[:, ['bmi*children', 'bmi', 'children']].head(10)

,bmi*children,bmi,children
0,130.661859,32.665465,3
1,59.597449,29.798725,1
2,32.722029,32.722029,0
3,115.289493,38.429831,2
4,59.283707,29.641854,1
5,113.357811,37.785937,2
6,130.180122,43.393374,2
7,64.560814,32.280407,1
8,57.731656,28.865828,1
9,101.612661,33.870887,2


In [ ]:
# qcut を用いて、px_height*px_widthを3分割する bmi*children_area を作成
train["bmi*children_area"] =  pd.qcut(train["bmi*children"], 3)
train[["bmi*children_area", "charges"]].groupby(["bmi*children_area"], as_index=False).mean().sort_values(by="bmi*children_area", ascending=True)

,bmi*children_area,charges
0,"(20.627, 33.61]",0.213483
1,"(33.61, 74.231]",0.311445
2,"(74.231, 283.744]",0.393996


In [ ]:
for dataset in combine:
    dataset.loc[ dataset["bmi*children"] <= 33.61, "bmi*children"] = 0
    dataset.loc[(dataset["bmi*children"] > 33.61) & (dataset["bmi*children"] <= 74.231),
                "bmi*children"] = 1
    dataset.loc[ dataset["bmi*children"] > 74.231, "bmi*children"] = 3
    dataset["bmi*children"] = dataset["bmi*children"].astype(int)

train = train.drop(["bmi*children_area"], axis=1)
combine = [train, test]

In [ ]:
# qcut を用いて、bmiを5分割する bmi_area を作成
train["bmi_area"] =  pd.qcut(train["bmi"], 5)
train[["bmi_area", "charges"]].groupby(["bmi_area"], as_index=False).mean().sort_values(by="bmi_area", ascending=True)


,bmi_area,charges
0,"(20.627, 28.233]",0.181250
1,"(28.233, 29.62]",0.246875
2,"(29.62, 33.017]",0.218750
3,"(33.017, 37.762]",0.403125
4,"(37.762, 47.291]",0.481250


In [ ]:

for dataset in combine:
    dataset.loc[ dataset['bmi'] <= 28.233, 'bmi'] = 0
    dataset.loc[(dataset['bmi'] > 28.233) & (dataset['bmi'] <= 29.62),
                'bmi'] = 1
    dataset.loc[(dataset['bmi'] > 29.62) & (dataset['bmi'] <= 33.017),
                'bmi'] = 2
    dataset.loc[(dataset['bmi'] > 33.017) & (dataset['bmi'] <= 37.762),
                'bmi'] = 3
    dataset.loc[(dataset['bmi'] > 37.762),'bmi'] = 4
    dataset["bmi"] = dataset["bmi"].astype(int)
train.tail(10)


,id,age,sex,bmi,children,smoker,region,charges,bmi*children,bmi_area
1590,1989,24,male,1,0,no,southeast,0,0,"(28.233, 29.62]"
1591,1990,53,male,2,0,no,northwest,0,0,"(29.62, 33.017]"
1592,1991,64,female,4,0,no,northwest,0,1,"(37.762, 47.291]"
1593,1992,24,female,2,1,no,southeast,0,1,"(29.62, 33.017]"
1594,1993,18,female,3,0,no,southeast,0,0,"(33.017, 37.762]"
1595,1994,52,male,2,0,no,southwest,0,0,"(29.62, 33.017]"
1596,1996,29,male,3,0,no,northwest,0,1,"(33.017, 37.762]"
1597,1997,56,female,4,1,no,southeast,0,3,"(37.762, 47.291]"
1598,1998,51,male,0,1,yes,northwest,0,1,"(20.627, 28.233]"
1599,1999,54,female,1,0,no,northwest,0,0,"(28.233, 29.62]"


In [ ]:

train = train.drop(["bmi_area"], axis=1)
combine = [train, test]


In [ ]:
for dataset in combine:
    dataset['sex'] = dataset['sex'].map( {'female': 1, 'male': 0} ).astype(int)

train.head()

,id,age,sex,bmi,children,smoker,region,charges,bmi*children
0,0,26,0,2,3,no,southeast,0,3
1,1,41,0,2,1,no,southwest,0,1
2,2,28,0,2,0,yes,northwest,1,0
3,3,20,1,4,2,no,southeast,0,3
4,4,45,1,2,1,no,northwest,0,1


In [ ]:
for dataset in combine:
    dataset['smoker'] = dataset['smoker'].map( {'no': 0, 'yes': 1} ).astype(int)

train.head()

,id,age,sex,bmi,children,smoker,region,charges,bmi*children
0,0,26,0,2,3,0,southeast,0,3
1,1,41,0,2,1,0,southwest,0,1
2,2,28,0,2,0,1,northwest,1,0
3,3,20,1,4,2,0,southeast,0,3
4,4,45,1,2,1,0,northwest,0,1


In [ ]:
for dataset in combine:
    dataset['region'] = dataset['region'].map( {'southeast': 0, 'northwest': 1,'southwest': 2,'northeast': 3} ).astype(int)

train.head()

,id,age,sex,bmi,children,smoker,region,charges,bmi*children
0,0,26,0,2,3,0,0,0,3
1,1,41,0,2,1,0,2,0,1
2,2,28,0,2,0,1,1,1,0
3,3,20,1,4,2,0,0,0,3
4,4,45,1,2,1,0,1,0,1


In [ ]:
#train = train.drop(["region"], axis=1)
#test = test.drop(["region"], axis=1)
#combine = [train, test]


In [ ]:
# 識別のために残しておいた id を落とす
# ただし、テストデータの id は提出時の index として用いるため保持
train = train.drop(["id"], axis=1)
#test = test.drop(["id"], axis=1)
combine = [train, test]
train.shape, test.shape

((1600, 8), (400, 8))

In [ ]:
X_train = train.drop("charges", axis=1)
y_train = train["charges"]
X_test =  test.drop("id", axis=1).copy()

In [ ]:
# warning (警告を非表示、例外化する)をインポート
import warnings
warnings.filterwarnings("ignore")
# LightGBM のインポート
import lightgbm as lgb

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn import metrics

In [ ]:
X_train_lgb, X_valid_lgb, Y_train_lgb, Y_valid_lgb = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [ ]:

lgb_params = {
    "objective" : "multiclass",
    "metric" : "multi_logloss",
    "num_class" : 3,
    "random_state": 777,
    "max_depth" : 7,
    "learning_rate" : 0.01,
}

lgtrain = lgb.Dataset(X_train_lgb, label=Y_train_lgb)
lgtest = lgb.Dataset(X_valid_lgb, label=Y_valid_lgb)

model_lgb = lgb.train(lgb_params,
                    lgtrain,
                    num_boost_round=1000,
                    valid_sets=[lgtrain, lgtest],
                    early_stopping_rounds=100,
                    verbose_eval=100,
                   )

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000054 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 70
[LightGBM] [Info] Number of data points in the train set: 1280, number of used features: 7
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score -0.244862
[LightGBM] [Info] Start training from score -2.073211
[LightGBM] [Info] Start training from score -2.392441
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until valida

In [ ]:
y_pred = model_lgb.predict(X_test).argmax(axis=1)

In [ ]:
X_test.shape

(400, 7)

In [ ]:
submit = pd.DataFrame({"index": test["id"],"charges": y_pred})

In [ ]:
submit.to_csv("./submit_lgb.csv", header=False, index=False)